# Buchhandelsdaten in Vufind als Grundlage für PDA (Patron Driven Aquisition) am MPIfG

Einbindung von freundlicherweise von Schweitzer Fachinformation zur Verfügung gestellten Daten.   
(Auswahl über passend konfigurierte Neuerscheinungsabfragen in unserem Kundenprofil).   

Das Jupyter Notebook arbeitet mit Python 3.8.10 und wurde mit Visual Studio Code 1.58.2 erstellt 


#### Arbeitsschritte im Code:

- Vorarbeiten
  - Notwendige Pandas Libraries aufrufen
  - Daten abholen und einlesen  
- Daten aufbereiten   
     - Dublettencheck innerhalb der Buchhandelsdaten    
     - Bestandsabgleich durch Abfragen auf dem Aleph-Server   
     - Aufbereitung und Einspielen der Antworten   
     - Erstellen der Exportdatei   


<hr>

## Vorarbeiten

### Pandas Libraries laden

In [1]:
import pandas as pd                                                # für das Arbeiten mit der CSV-Datei
import urllib.request                                              # für das Abrufen der URL
import requests                                                    #für die Bestandsabfragen 
pd.options.mode.chained_assignment = None                          # default='warn' abschalten beim Beschreiben der neuen Spalten
import time                                                        # für das Schreiben des Datums Logdatei und Excel-Export und Arbeiten mit dem Erscheinungsdatum
import numpy as np                                                 # für das Bearbeiten von Spalten

### Prüfung, ob die Verbindung zum Aleph-Server für Abfragen korrekt funktioniert:

    Nur zugelassene IPs können diese Schnittstelle abfragen.

In [2]:
test= "http://aleph.mpg.de/X?op=find&base=ges01&request=IBS=9783482648434"

reply = requests.get(test).text
a = reply.find("Forbidden")
b =  reply.find("?xml")

if (a > 50):
    print("Es gibt ein Problem mit dem Server")
if (b == 1):
    print("Der Server antwortet korrekt")

Der Server antwortet korrekt


In [1]:
print(reply)

NameError: name 'reply' is not defined

### Datensätze abholen und einlesen

In [3]:
url = "https://content.schweitzer-online.de/static/content/export/mpifg/export.csv"  # Abruf, der von Schweitzer zur Verfügung gestellten Daten
checkout_file = "./input/export.csv"  
urllib.request.urlretrieve(url, checkout_file)

('./input/export.csv', <http.client.HTTPMessage at 0x2cca94383d0>)

In [4]:
df = pd.read_csv('./input/export.csv', encoding = 'UTF-8', sep=';' , keep_default_na=False) # muss encoding angeben und Trennzeichen, NaN (= leere Werte) direkt beim Import entfernen, da sie später Probleme machen

### LOG-Datei für den Prozess, zur Dokumentation des Imports und als Kontrollanzeige hier


In [5]:
x = df.shape[0]
print('Anzahl der enthaltenen Datensätze:', x)
print('vorhandene ISBNs:', df["isbn_ean"].shape[0])

timestr = time.strftime('%d.%m.%Y - %H:%M')

with open ('./log/pda_import_log.txt', 'a') as log:                                                  # Da diese Log-Datei nicht unmittelbar gebraucht wird, hier fortlaufendes Schreiben in eine Datei
    log.write('Logdatei PDA-Import vom ')
    log.write(timestr)
    log.write('\n------------------------------------------\n')
    log.write('Gelieferte Datensätze:             ' + str(x))

Anzahl der enthaltenen Datensätze: 3202
vorhandene ISBNs: 3202


<hr>

# Daten aufbereiten

<hr>

## Dublettencheck innerhalb der Buchhandelsdaten 

Aufgaben im Rahmen des Dublettencheck:
1. Dublettenkontrolle anhand von Titel, Untertitel und Autor 
   - Zunächst Behebung der unsauberen Titel / Untertitel-Trennung für korrekteren Abgleich
   - Trennung der Datensätze in Dubletten und "Einzeltitel"
     - Einzeltitel werden direkt für Bestandsprüfung vorgemerkt
     - Dubletten werden auf neueste Version reduziert und diese der Bestandsprüfungsdatei hinzugefügt

### Die Dublettenkontrolle Anhand von Titel, Untertitel und Autor

*Entfernen von Untertiteln aus der Titelspalte, Extrahieren von Untertiteln und Abgleich mit Untertitelspalte und Schreiben der vorhandenen Informationen in neue Untertitel-Spalte.   
Durch diese Spalte werden ca. 1/3 mehr Dubletten erkannt, als ohne die Bereinigung. *

In [6]:
neu = df["title"].str.split(':', n = 1, expand = True)  #Titel am 1. Doppelpunkt splitten und getrennt in neue Felder schreiben
df["title_sep"]= neu[0]
df["subtitle_sep"]= neu[1]

df["subtitle_sep"] = df["subtitle_sep"].replace(np.nan, '', regex=True) #NaN-Werte stören, darum raus damit ...

In [7]:
comparison = np.where(df["subtitle"] == df["subtitle_sep"], '', df["subtitle"])    # Abgleich - wenn in beiden das Gleiche steht, dann ursprüngliches "Subtitle"-Feld nehmen
df["subtitle_comparison"] = comparison  

comparison2 = np.where(df["subtitle"] < df["subtitle_sep"], df["subtitle_sep"], '') # Wenn nur in "subtitle_sep" Infos stehen, diese übernehmen, das ist noch nicht ganz sauber, da hier manchmal anderes steht als in "subtitle"
df["subtitle_comparison2"] = comparison2 

df["subtitle_all"] = df["subtitle_comparison"]+df["subtitle_comparison2"]          # Beide Informationen in neuer Subtitle-Spalte zusammenführen

In [8]:
df["short_title"] = df["title_sep"] + ' ' + df["subtitle_all"] + ' / ' + df["contributor"]  # aus den bereinigten Daten einen Kurztitel erzeugen, der dann für den Dublettencheck verwendet wird

In [9]:
df_dubletten = df.groupby("short_title").filter(lambda g: (g.nunique() >1).any()) # schreibt alle mehrfach vorhandenen Titel in ein eigenes Datenframe

In [10]:

df_dubl_einspielen = df_dubletten.sort_values(by=["short_title", "publication_year"], ascending =False).drop_duplicates(subset=["short_title"], keep='first')   # sortiert Dubletten nach Jahr und schreibt den jeweils ersten (= neuesten) Eintrag in neues Dataframe


In [11]:
df_ohne_dubletten = df.drop_duplicates("short_title", keep=False)       #durch ""keep=False" werden alle nicht-Dubletten rausgezogen

In [12]:
df_einspielen = df_ohne_dubletten.append(df_dubl_einspielen)                    # die ausgewählten Dubletten und alle Nicht-Dubletten werden in ein Datenframe zusammengeführt
df_einspielen.reset_index(inplace=True)

<hr>

## Datensätze am Bestand abgleichen

*URLs für die Abfrage über den X-Server unseres Bibliothekssystems werden erzeugt und über die ISBN eine Abfrage auf Bestand gemacht. Die Abfrage funktioniert nur für zugelassene IPs (darum oben die Prüfung).  
Für die Abfrage in unseren Bestand ist die ISBN sehr gut, da in den Titeldaten alle im Buch befindlichen ISBNs - auch die anderer Ausgabeformen - mit übernommen sind. Beim MPG-Ebooks Katalog handelt sich um Daten von Verlagen, die sich in ihrer Qualität und Informationsumfang sehr unterscheiden. Hier wird noch zu prüfen sein, inwieweit ein anderer Abfragemechansimus gewählt werden muss.*  

#### URLs für unseren Bestand erzeugen

*Es funktionierte nicht, dass die URLs an die vorhandenen ISBNs einfach so angefügt werden, darum der Workaround mit einem Platzhalter, der sich dann über replace vom richtigen Link überschreiben ließ.*

In [13]:
df_einspielen["url_ges"] = df_einspielen["isbn_ean"].apply(lambda x: f"ges_link{x}".replace('ges_link','http://aleph.mpg.de/X?op=find&base=ges01&request=IBS='))

#### URLs für Ebooks-Katalog erzeugen 

In [14]:
df_einspielen["url_ebx"] = df_einspielen["isbn_ean"].apply(lambda x: f"ebx_link{x}".replace('ebx_link', 'http://aleph.mpg.de/X?op=find&base=ebx01&request=IBN='))

#### Abfragen beim Server

##### Zunächst für die Daten des MPIfG 

*Vorgehensweise: Abfrage und Sammeln der Antworten in einer Datei, diese Antworten werden dann in Ausdrücke "übersetzt" - "vorhanden" und "neu" und diese Daten in eine Spalte ins Dataframe zur weiteren Auswertung übertragen.   
Schwierigkeit hier war, die Sammlung der Antworten zu den einzelnen Titeln, um sie in das Datenframe einzuspielen. Der störende XML-Header der Antworten wird erst gar nicht in die Datei geschrieben.   *   

    Hinweis: 
    Vielleicht werden wir mittelfristig, um die Anzahl der Abfragen auf dem Server zu reduzieren, nach dem Abgleich mit dem eigenen Bestand, die Titel auf "Neue" reduzieren. Im Moment möchten wir jedoch die Abfrage prüfen und darum einen Blick auf die Bestandsabfrage machen. Aus diesem Grund wird unten auch eine Excel-Exportdatei dieser Titel erstellt. Ebenso ist die Dublettenprüfung vor den Bestandsabfragen denkbar, aber im Moment noch für die Kontrolle der Einstellungen in dieser Reihenfolge. 

In [15]:
with open('./input/server_responses', 'w') as fn:  
    for url in df_einspielen["url_ges"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses', 'r') as f:
    with open('./input/server_responses_transfered', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf = pd.read_fwf('./input/server_responses_transfered', names=["Abfrage_ges"])
df_result = pd.concat([df_einspielen, df_fwf], axis=1)             

    Randnotiz: 
    Bei 2400 Titels brauchte der Abgleich ca 350 Sekunden

In [16]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind, wird mittelfristig rausfallen
x = df_einspielen.shape[0]
y = df_fwf.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 2826 
Anzahl der Antworten vom Server:    2826


##### Datenabgleich mit dem Bestand des MPG Ebooks-Katalog

*Vorgehensweise analog Bestandsabfrage MPI.*

In [17]:
with open('./input/server_responses_ebx', 'w') as fn:  
    for url in df_result["url_ebx"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses_ebx', 'r') as f:
    with open('./input/server_responses_transfered_ebx', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ebx = pd.read_fwf('./input/server_responses_transfered_ebx', names=["Abfrage_ebx"])
df_result2 = pd.concat([df_result, df_fwf_ebx], axis=1)

In [18]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind
x = df_result2.shape[0]
y = df_fwf_ebx.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 2826 
Anzahl der Antworten vom Server:    2826


<hr>

## Exportvorbereitungen

### Zunächst ein Blick auf die vorhandenen Titel

##### Zur Qualitätskontrolle und aus Interesse, werden die als vorhanden gekennzeichneten Titel in eine datierte Excel-Tabelle geschrieben 

In [19]:
df_nicht_einspielen = df_result2.drop(df_result2[(df_result2["Abfrage_ebx"]== 'neu') & (df_result2["Abfrage_ges"] == 'neu')].index) 
# alle Titel rausholen, die in einer der beiden Datenbanken vorhanden waren

In [20]:
date = time.strftime("%Y_%m_%d")                                              # Zeit erfassen für Dateibenennung

df_nicht_einspielen["object_id"] = df_nicht_einspielen.object_id.astype(str)  # wandelt die spalte von Int64 zu Object um, so dass es in Excel korrekt eingelesen wird
df_nicht_einspielen["isbn_ean"] = df_nicht_einspielen.isbn_ean.astype(str)
df_nicht_einspielen = df_nicht_einspielen.drop(columns=["url_ebx", "url_ges", "cover", "title_sep", "subtitle_comparison", "subtitle_comparison2", "subtitle_all", "subtitle_sep"]) # unnötige Spalten entfernen

df_nicht_einspielen.to_excel('./output/Vorhandene_Titel_'+date+'.xlsx', engine='xlsxwriter')

### Jetzt Extraktion der Titel zum Einspielen:

In [21]:
df_aleph_einspielen = df_result2.loc[((df_result2["Abfrage_ebx"]== 'neu') & (df_result2["Abfrage_ges"] == 'neu'))]

#### Für die Logdatei Ermittlung verschiedener Zahlen und hier zur direkten Ansicht ausgegeben

In [22]:
#Kontrollmechanismus, ob für alle Titel auch Treffer da sind
x = df_result2.shape[0]  #Ursprungszahl, eigentlich oben schon berechnet ...
a = df_ohne_dubletten.shape[0]
c = df_dubletten.shape[0]
b = df_dubl_einspielen.shape[0] #Auswahl der neuen Treffer
y = df_nicht_einspielen.shape[0] #vorhandene
z = df_aleph_einspielen.shape[0]
d = a + b
print('Kleine Statistik:\n=====================================',
    '\nGelieferte Datensätze:           ', x, 
    '\n-------------------------------------',
    '\nSätze ohne Dubletten:            ', a,
    '\n    Dubletten:       ', c, 
    '\n    Davon zum Einspielen:        ', b,
    '\nAm Bestand abzugleichen:         ', d,
    '\n----------------------------------------------------------------',
    '\nVorhandene Titel, die nicht in Aleph exportiert werden:   ', y,
    '\nTatsächlich eingespielt werden:                           ', z,
    '\n----------------------------------------------------------------',)

#hier entsprechende Einträge für die Log-Datei

with open ('./log/pda_import_log.txt', 'a') as log:
    log.write("\nSätze ohne Dubletten:              " + str(a))
    log.write("\n   Dubletten:              " + str(c))
    log.write("\n   Auswahl zum Einspielen:         " + str(b))
    log.write("\n--------------------------------------------\n")
    log.write("\nAm Bestand abzugleichen:           " + str(d))
    log.write("\n        davon vorhanden:           " + str(y))
    log.write("\n--------------------------------------------\n")
    log.write("\nEingespielt werden:                " + str(z))
    log.write("\n============================================================\n\n")

Kleine Statistik:
Gelieferte Datensätze:            2826 
------------------------------------- 
Sätze ohne Dubletten:             2472 
    Dubletten:        730 
    Davon zum Einspielen:         354 
Am Bestand abzugleichen:          2826 
---------------------------------------------------------------- 
Vorhandene Titel, die nicht in Aleph exportiert werden:    104 
Tatsächlich eingespielt werden:                            2722 
----------------------------------------------------------------


<hr>

## Aufbereiten der Exportdatei

#### Zielformat für das Einspielen in Aleph:

    000000001 LDR   L -----nM2.01200024------h              
    000000001 020   L $$a (object_id))
    000000001 030   L $$aaz||rrrza||||
    000000001 051   L $$am|||||||
    000000001 070   L $$aSchweitzer
    000000001 077   L $$aMonographie
    000000001 078   L $$aSchweitzer
    000000001 082   L $$azum Bestellen
    000000001 100   L $$a (contributor_1)
    000000001 104   L $$a (contributor_2)
    000000001 108   L $$a (contributor_3)
    000000001 331   L $$a (title_sep)
    000000001 335   L $$a (subtitle_all)
    000000001 403   L $$a (edition_number / edition_text)  #noch prüfen, was besser zu verwenden ist 
    000000001 419   L $$b (publisher) $$a (date_combined)
    000000001 433   L $$a (pages)
    000000001 451   L $$a (series)
    000000001 520   L $$a (thesis)
    000000001 540   L $$a (isbn_ean)
    000000001 656   L $$a (cover)
    000000001 750   L $$a (description)
    000000001 655   L $$zOrder me$$umailto:bib@mpifg.de?subject=Bestellwunsch        
    
Anmerkung zum Feld 655: die URL wird NACH dem Einspielen in Aleph mit der Datensatz-ID angereichert (siehe Juypter-Notebook "Link-Anreicherung"), um einen klaren Bestellink für den Kaufvorschlag zu haben

*Hierfür werden immer die Feldbenennung und bestimmte Codierungen VOR den Inhalt - in Klammern de Bezeichnung der entsprechenden Spalte - gesetzt, bzw. erfoderliche Felder komplett neu hinzugefügt.   
Am Anfang jeder Zeile braucht Aleph eine 9-Stellige eindeutige Zahl pro Titel.*   

*Manchmal ließ sich der Inhalt einer Spalte direkt in die Datei schreiben, manchmal musst die Spalte zuvor über apply aufbereitet werden.* 

In [23]:
df_aleph_einspielen["020"] = df_aleph_einspielen["object_id"].apply(lambda x: f"020   L $$a{x}") 
del df_aleph_einspielen["object_id"]                                                                      #um das df nicht unnötig anwachsen zu lassen, jeweils alte Spalte löschen

#### Besondere Aufbereitung der Personendaten

*Da bis zu 3 Personen in einer Spalte zu finden sind, werden diese im Discovery nicht getrennt suchbar, darum werden sie gesplittet. Für die Dublettenkontrolle hat sich das als irrelevant erwiesen, darum erfolgt dieser Schritt erst hier.*

In [24]:
person = df_aleph_einspielen["contributor"].str.split(';', expand=True)                         #Für saubere Daten, die Autorenangabe splitten und in getrennte Felder schreiben

df_aleph_einspielen["contributor_1"]= person[0]
df_aleph_einspielen["contributor_2"]= person[1]
df_aleph_einspielen["contributor_3"]= person[2]

df_aleph_einspielen["contributor_1"]= df_aleph_einspielen["contributor_1"].replace(np.nan, '', regex=True)
df_aleph_einspielen["contributor_2"]= df_aleph_einspielen["contributor_2"].replace(np.nan, '', regex=True)
df_aleph_einspielen["contributor_3"]= df_aleph_einspielen["contributor_3"].replace(np.nan, '', regex=True)


#### Besondere Aufbereitung des Erscheinungsdatum und Erscheinungsjahres

*In der Auswahl unserer Titel befinden sich auch im Erscheinen befindliche Titel der kommenden Monate. Diese Information möchten wir gerne im Discovery sichtbar machen. Hierfür bleibt uns nur Aleph-Feld 419c, das dem Erscheinungsjahr vorbehalten ist.   
Wunsch ist es: Wenn des Erscheinungsdatum weiter als 10 Tage weg vom heutigen Datum ist, soll das komplette Datum angezeigt werden, ansonsten nur das Erscheinungsjahr.*

Zur Umsetzung muss die Spalte "publication_date" in ein Datum verwandelt werden und nach den genannten Kriterien unterschiedlich angezeigt

In [25]:
today = int(time.strftime('%Y%m%d'))
df_aleph_einspielen["coming_soon"] = np.where(df_aleph_einspielen["publication_date"].astype(int) > today+10, df_aleph_einspielen["publication_date"], np.nan) #zieht die über 10 Tage raus, brauchen nan für Umwandlung in Datum

In [26]:
df_aleph_einspielen["publication_date_soon"] = df_aleph_einspielen["coming_soon"].astype(str).str.replace('00','01')  # es gibt invalide Daten wie 20210800, darum diese erst auf den 1. des Monats geschoben.
df_aleph_einspielen["coming_soon"] = pd.to_datetime(df_aleph_einspielen["publication_date_soon"], format='%Y%m%d.0', errors='ignore').astype(str).str.replace('NaT','') # Formatumwandlung, "ignore", da es weitere Fehlerhafte Daten in falscher Reihenfolge gibt

In [27]:
df_aleph_einspielen["published"] = np.where(df_aleph_einspielen["coming_soon"] == '', df_aleph_einspielen["publication_year"], '') #Auslesen und Kombinieren der Daten
df_aleph_einspielen["date_combined"] = df_aleph_einspielen["published"]+df_aleph_einspielen["coming_soon"]
#df_aleph_einspielen['date_combined']

In [28]:
df_aleph_einspielen["419b"] = df_aleph_einspielen["publisher"].apply(lambda x: f"419   L $$b{x}") 
df_aleph_einspielen["419c"] = df_aleph_einspielen["date_combined"].apply(lambda x: f"$$c{x}")                  

df_aleph_einspielen["419"] = df_aleph_einspielen["419b"]+df_aleph_einspielen["419c"]                           #Für die Korrekte Eingabe brauche ich Verlag und Jahr in einer Spalte

In [29]:
df_aleph_einspielen["403"] = df_aleph_einspielen["edition_text"].apply(lambda x: f"403   L $$a{x}") 
df_aleph_einspielen["433"] = df_aleph_einspielen["pages"].apply(lambda x: f"433   L $$b{x}")
df_aleph_einspielen["451"] = df_aleph_einspielen["series"].apply(lambda x: f"451   L $$b{x}") 
df_aleph_einspielen["520"] = df_aleph_einspielen["thesis"].apply(lambda x: f"520   L $$a{x}") 
df_aleph_einspielen["540"] = df_aleph_einspielen["isbn_ean"].apply(lambda x: f"540   L $$a{x}") 
df_aleph_einspielen["656"] = df_aleph_einspielen["cover"].apply(lambda x: f"656   L $$u{x}") 
df_aleph_einspielen["750"] = df_aleph_einspielen["description"].apply(lambda x: f"750   L $$a{x}") 

#### Für das Durchzählen der Titel braucht es eine neue Spalte

In [30]:
# hier entsteht eine neue Spalte mit Zahlen ab 1 durchgehend gezählt, die für den korrekten Import der Daten in Aleph nötig ist
x = df_aleph_einspielen.shape[0]   
df_aleph_einspielen["id"] = range(1,x+1)                                                       #Notwendig ist die Zählung ab 1, da Aleph sonst nicht korrekt einließt
df_aleph_einspielen["id"] = df_aleph_einspielen["id"].apply(lambda x: f"{x:09d}")              #Die Zahl muss 9-Stellig aufgefüllt werden

<hr>

### Vorbereitungen abgschlossen, jetzt das Schreiben der Datei im Aleph-Sequential-Format:

In [31]:
with open("./output/pda_ges02", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_einspielen.index:
        fa.write(df_aleph_einspielen["id"][i]+' LDR   L -----nM2.01200024------h'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["020"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 030   L $$aaz||rrrza||||'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 051   L $$am|||||||m|||||||'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 077   L $$aMonographie'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 078   L $$aSchweitzer'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 082   L $$azum Bestellen'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 100   L $$a'+df_aleph_einspielen["contributor_1"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 104   L $$a'+df_aleph_einspielen["contributor_2"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 108   L $$a'+df_aleph_einspielen["contributor_3"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 331   L $$a'+df_aleph_einspielen["title_sep"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 335   L $$a'+df_aleph_einspielen["subtitle_all"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["403"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["419"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["433"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["451"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["520"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["540"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 655   L $$zOrder me$$umailto:bib@mpifg.de?subject=Bestellwunsch'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["656"][i]+'$$3Cover\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["750"][i]+'\n')

*Anschließend Import der Datei "pda_ges02" in Aleph und Anreicherung mit den Bestelllinks über das Skript in Aleph_Link-Anreicherung.ipynb*